In [1]:
pip install FastKmedoids

Note: you may need to restart the kernel to use updated packages.


In [2]:
from FastKmedoids import package_status

In [3]:
package_status()

This package is under construction...
